In [1]:
import numpy as np
import pandas as pd
import json
import pymongo
import re
from itertools import cycle
import glob
import os
from more_itertools import split_before
from importlib import reload
import retrosheetParser
pd.set_option("display.max_rows", None)

In [62]:
reload(retrosheetParser)

<module 'retrosheetParser' from 'C:\\Users\\Gregory\\Documents\\PythonScripts\\retrosheetParser.py'>

In [2]:
#mongoclient = pymongo.MongoClient()

In [3]:
#db = mongoclient['baseball']

In [63]:
dirname = "C:/Users/Gregory/Documents/Baseball2019Events/"
eventfiles = [f for f in os.listdir(dirname) if re.search("\.EV(N|A)$",f)]

In [64]:
events = []
for filename in eventfiles:
    with open(dirname + filename) as f:
        content = f.readlines()
        events = events + content
events = [line.strip('\n') for line in events]

In [65]:
len(events)

407693

In [66]:
content = list(split_before(events, lambda x: x.startswith("id,")))

In [67]:
d = [retrosheetParser.RetrosheetEventFormatter(game) for game in content]

In [68]:
Plays2019 = pd.concat([(d[i]['plays'].assign(Game = d[i]['id'])
 .set_index(['Game',d[i]['plays'].index])
 .rename_axis(['Game','PlayNum'])) for i in range(len(d))])

In [69]:
Plays2019.loc[('ANA201904040',7),'Comment']

'Umpire review of ball hit by Guzman and ruled that it hit foul pole - home run'

In [70]:
Plays2019.head(100)

Type Inning Team  PlayerID Count PitchSequence  \
Game         PlayNum                                                   
ANA201904040 0        play      1    0  choos001    22         FBBCH   
             1        play      1    0  odorr001    12          BFSS   
             2        play      1    0  andre001    12          CCBX   
             3         com                                             
             4        play      1    0  gallj002    00             X   
             5        play      1    0  cabra002    30          BBBB   
             6         com                                             
             7        play      1    0  guzmr001    12          SSBX   
             8         com                                             
             9         com                                             
             10       play      1    0  forsl001    12          BCSS   
             11       play      1    0  kinei001    01            CX   
             12       play      1    0  deshd002    12        CB11FT   
             13       play      1    1  calhk001    31         BFBBX   
             14       play      1    1  troum001    00             X   
             15       play      1    1  bourj002    32     CBF1B*B*B   
             16        com                                             
             17       play      1    1  simma001    02          FLFS   
             18       play      1    1  pujoa001    31         CBBBB   
             19       play      1    1  lastt001    01            CX   
             20       play      2    0  choos001    11           BCX   
             21       play      2    0  odorr001    12          CBSS   
             22       play      2    0  andre001    12        11CBSC   
             23       play      2    0  gallj002    22        S*BSBX   
             24       play      2    1  lucrj001    12          BCFX   
             25       play      2    1  goodb001    02           CSS   
             26       play      2    1  cozaz001    12         CSFBX   
             27       play      3    0  cabra002    00                 
             28       play      3    0  cabra002    00                 
             29        com                                             
             30       play      3    0  cabra002    02         ..CFX   
             31       play      3    0  guzmr001    12          CBFX   
             32       play      3    0  forsl001    02           CFX   
             33       play      3    0  kinei001    32        BBFFBX   
             34       play      3    0  deshd002    12         CFBFH   
             35        com                                             
             36       play      3    0  choos001    11           BSX   
             37       play      3    1  calhk001    32       SBBFBFC   
             38       play      3    1  troum001    31         BBBCX   
             39       play      3    1  bourj002    22         BCSBX   
             40       play      3    1  fletd002    22         CBBCX   
             41       play      3    1  pujoa001    31         BBFBB   
             42        com                                             
             43       play      3    1  lastt001    21          BCBB   
             44       play      3    1  lastt001    32       BCBB.CX   
             45       play      4    0  odorr001    32       CSBBBFX   
             46       play      4    0  andre001    12          SBFX   
             47       play      4    0  gallj002    10           B1X   
             48       play      4    1  lucrj001    12          CBFX   
             49       play      4    1  goodb001    30          BBBB   
             50       play      4    1  cozaz001    32      BCBC1FBX   
             51        com                                             
             52       play      4    1  calhk001    00                 
             53       play      4    1  calhk001    12        

In [71]:
TestData = (Plays2019
 .query("Type == 'play'")
 #.head(100)
 .assign(playSplit = lambda x: x.Play.transform(retrosheetParser._playSplitter))
 .assign(subevents = lambda x: x.playSplit.apply(lambda z: retrosheetParser._subevent_splitter(z['event'])))
 .assign(subeventParsed = lambda x: x.subevents.apply(lambda z: [retrosheetParser._subeventParser(j) for j in z]))
 .assign(basicPlayDesc = lambda x: x.subeventParsed.transform(retrosheetParser.basicPlayDesc))
 .assign(fullPlayDesc = lambda x: x.subeventParsed.transform(retrosheetParser.fullPlayDesc))
 .assign(playersOut = lambda x: x.apply(lambda z: retrosheetParser.playersOut(z.playSplit,z.subeventParsed), axis = 1))
)

In [72]:
(TestData
 .assign(eventLength = lambda x: x.subevents.apply(lambda z: len(z)))
 .query("eventLength == 2",engine='python')
 .head(100))

Type Inning Team  PlayerID Count PitchSequence  \
Game         PlayNum                                                   
ANA201904040 19       play      1    1  lastt001    01            CX   
             69       play      6    0  cabra002    22        CBFBFX   
             86       play      7    1  lucrj001    12          CFBX   
             106      play      9    1  bourp001    22        BCSBFS   
ANA201904050 3        play      1    0  mazan001    02           CSS   
             9        play      2    0  guzmr001    12         BCCFX   
             26       play      3    1  troum001    21          BCBX   
ANA201904060 36       play      4    0  mazan001    22  SF1*BFB1FFFX   
             88       play      8    1  troum001    32     CB1B11FBS   
ANA201904070 42       play      4    0  cabra002    00             X   
             64       play      6    0  cabra002    11           BCX   
             75       play      7    0  deshd002    22        CBC*BX   
             80       play      7    1  fletd002    12          CBFX   
ANA201904080 8        play      1    1  simma001    00             X   
             24       play      3    1  troum001    32       F*BBCBS   
             33       play      4    0  arcio002    01            CX   
ANA201904090 72       play      6    0  braur002    01            SX   
ANA201904100 2        play      1    0  braur002    32      1BB1FBFX   
             32       play      3    1  pujoa001    12          SFBX   
             48       play      5    0  pereh001    12          BSFX   
ANA201904180 15       play      2    1  smitk002    01            CX   
             47       play      5    1  simma001    22         BSBFX   
ANA201904190 73       play      8    1  bourj002    02        ..CS1X   
ANA201904200 91       play      8    1  lastt001    22        .FCBBX   
ANA201904210 30       play      4    1  smitk002    22      B1BCCF1X   
             80       play      8    1  bourj002    00            .X   
ANA201904220 14       play      2    1  cozaz001    00             X   
             67       play      8    1  bourj002    01          .1CX   
             75       play      9    1  simma001    11           SBX   
             107      play     12    1  lucrj001    12  .B1C1FFFF11S   
ANA201904230 24       play      3    1  troum001    12          CBFX   
             90       play      8    1  smitk002    21         FB*BX   
ANA201904240 71       play      7    1  lastt001    22      1BBFFFFX   
ANA201904250 13       play      2    0  urshg001    12        FSF*BX   
             37       play      4    1  bourj002    32     C*BCB*B1C   
             80       play      7    1  troum001    32      *BB.BCCB   
ANA201904300 12       play      2    1  lucrj001    01            CX   
             86       play      9    0  tellr001    00           .1X   
ANA201905010 41       play      4    1  simma001    00             X   
             77       play      7    1  lucrj001    22       CF*B1BS   
             95       play      9    0  drurb001    12        ..CCBS   
ANA201905020 55       play      6    0  drurb001    12       FFBFFFX   
ANA201905040 35       play      4    0  sprig001    11         .S*BX   
             75       play      7    1  pujoa001    12       CFFFFBX   
ANA201905050 14       play      2    0  chirr001    11          *BCX   
ANA201905170 17       play      2    1  fletd002    11          C*BX   
             40       play      5    0  lopen001    32     C1FB1*BBT   
             44       play      5    1  pujoa001    32      *BCFB*BX   
             85       play      8    0  ohear001    32     CFB.*B*BB   
ANA201905200 3        play      1    0  rosae001    02           CSX   
ANA201905240 84       play      8    1  lastt001    02         1CFFX   
ANA201905250 29       play      4    1  pujoa001    12          CBFX   
             39       play      5    1  garnd001    22         CBFBX   
             44       play      6    0  mazan001    02        

In [73]:
(TestData
 .assign(OutsOnPlay = lambda x: x.playersOut.apply(len))
 .assign(OutsInInning = lambda x: x.groupby(['Game','Inning','Team'])['OutsOnPlay'].transform(sum))
 .assign(Inning = lambda x: x.Inning.transform(int))
 .query('OutsInInning != 3')
 .query("Inning < 9")
 .head(100)
 .loc[:,['Inning','Team','Play','Comment','playersOut','OutsOnPlay','OutsInInning']])

Inning Team                               Play  \
Game         PlayNum                                                   
BAL201906250 30            3    1                               53/G   
             31            3    1                                  K   
             32            3    1                                  W   
             33            3    1           D9/G+.1-H;BX3(E9)(95/TH)   
BOS201904130 70            7    0                               S9/G   
             71            7    0                        PO1(E1).1-2   
             72            7    0                                  W   
             73            7    0                                  K   
             74            7    0               E3/G.2-H(NR)(UR);1-3   
             76            7    0  FC5/G.3-H(RBI)(UR);1X2(54)(E5/TH)   
             80            7    0                                8/L   
NYA201904120 74            7    0                                 NP   
             75            7    0                                7/F   
             76            7    0                             HR/8/F   
             77            7    0                             HR/9/L   
NYA201905050 77            8    1                                 NP   
             79            8    1                                  W   
             80            8    1                             PB.1-2   
             81            8    1                             WP.2-3   
NYN201908220 59            8    1                                 NP   
             60            8    1                                 NP   
             61            8    1                                  W   
             62            8    1                          S9/G+.1-3   
OAK201905090 35            5    0               S2/BG.BX3(95)(E2/TH)   
             36            5    0                               63/G   
             37            5    0                                  K   

                                                                Comment  \
Game         PlayNum                                                      
BAL201906250 30                                                           
             31                                                           
             32                                                           
             33                                                           
BOS201904130 70                                                           
             71                                                           
             72                                                           
             73                                                           
             74                                          On-field Delay   
             76       play originally scored as fielder's choice wit...   
             80                                                           
NYA201904120 74                                                           
             75                                                           
             76                                                           
             77                                    Game called for rain   
NYA201905050 77                                          On-field Delay   
             79                                                           
             80                                                           
             81                                    Game called for rain   
NYN201908220 59                                                           
             60                                                           
             61                                                           
             62                                    Game called for rain   
OAK201905090 35                                                           
             36                                        

In [13]:
#event_collection = db['events']

In [22]:
#event_collection.insert_one(d)

In [123]:
#event_collection.find_one({"id":"BOS201904090"})

In [10]:
plays = d[0]['plays']

In [11]:
def baseRunnersResult(baseRunners,play,batter):
    baseRunners2 = baseRunners.copy()
    eventAdvances = re.split('\.',play)
    if len(eventAdvances) == 2:
        eventAdvances = re.split(';',eventAdvances[1])
    else:
        return(baseRunners2)
    for adv in re.findall("[1-3]-[1-3]",eventAdvances):
        pos = re.split("-",adv)
        baseRunners2[pos[1]] = baseRunners[pos[0]]
        if baseRunners2[pos[1]] == baseRunners2[pos[0]]:
            baseRunners2[pos[0]] = None
    batterAdvance = re.findall("B-[1-3]",eventAdvances)
    if len(batterAdvance):
        pos = re.split("-",batterAdvance)
        baseRunners2[pos[1]] = batter
    elif re.match()

SyntaxError: invalid syntax (<ipython-input-11-2f9ee36e9c9a>, line 17)

In [28]:
baseRunners = {'1':None,'2':'andre001','3':None}
baseRunners2 = baseRunners.copy()
baseRunners2['3'] = baseRunners['2']; baseRunners2['2'] = None;
print(baseRunners)
print(baseRunners2)

{'1': None, '2': 'andre001', '3': None}
{'1': None, '2': None, '3': 'andre001'}


In [15]:
re.split?

Signature: re.split(pattern, string, maxsplit=0, flags=0)
Docstring:
Split the source string by the occurrences of the pattern,
returning a list containing the resulting substrings.  If
capturing parentheses are used in pattern, then the text of all
groups in the pattern are also returned as part of the resulting
list.  If maxsplit is nonzero, at most maxsplit splits occur,
and the remainder of the string is returned as the final element
of the list.
File:      c:\users\gregory\anaconda3\lib\re.py
Type:      function


In [24]:
play = "S8/L.2XH(82/TH);B-2(TH)"
#play = 'K'
s1 = re.split('\/',play,maxsplit=1)
s2 = re.split('\.',play)
event = s1[0]
if len(s2) == 2:
    adv = s2[1]
else:
    adv = ""
if len(s1) == 1:
    mod = ""
elif len(s2) == 2:
    mod = s1[1][:-(len(adv) + 1)]
else:
    mod = s1[1]
print("Event: " + event)
print("Modifier: " + mod)
print("Advance: " + adv)
# event = s1[0]
# if len(s1) == 2:
#     event = s1[0]
#     s2 = re.split('\.',s1[1])
#     if len(s2) == 2:
len('whatever')

Event: S8
Modifier: L
Advance: 2XH(82/TH);B-2(TH)


8

In [19]:
(plays
 .assign(PlayDesc = lambda x: x.Play.transform(retrosheetParser.playParser))
 .assign(OutsOnPlay = lambda x: x.PlayDesc.transform(retrosheetParser.outsOnPlay))
 .assign(RunsScored = lambda x: x.Play.transform(retrosheetParser.runsScored))
 .assign(Game = 'Whatever')
 .set_index(['Game',plays.index])
 .rename_axis(['Game','PlayNum'])) #query('PlayDesc == ""')

Inning Team  PlayerID Count PitchSequence  \
Game     PlayNum                                             
Whatever 0            1    0  choos001    22         FBBCH   
         1            1    0  odorr001    12          BFSS   
         2            1    0  andre001    12          CCBX   
         3            1    0  gallj002    00             X   
         4            1    0  cabra002    30          BBBB   
         5            1    0  guzmr001    12          SSBX   
         6            1    0  forsl001    12          BCSS   
         7            1    0  kinei001    01            CX   
         8            1    0  deshd002    12        CB11FT   
         9            1    1  calhk001    31         BFBBX   
         10           1    1  troum001    00             X   
         11           1    1  bourj002    32     CBF1B*B*B   
         12           1    1  simma001    02          FLFS   
         13           1    1  pujoa001    31         CBBBB   
         14           1    1  lastt001    01            CX   
         15           2    0  choos001    11           BCX   
         16           2    0  odorr001    12          CBSS   
         17           2    0  andre001    12        11CBSC   
         18           2    0  gallj002    22        S*BSBX   
         19           2    1  lucrj001    12          BCFX   
         20           2    1  goodb001    02           CSS   
         21           2    1  cozaz001    12         CSFBX   
         22           3    0  cabra002    00                 
         23           3    0  cabra002    00                 
         24           3    0  cabra002    02         ..CFX   
         25           3    0  guzmr001    12          CBFX   
         26           3    0  forsl001    02           CFX   
         27           3    0  kinei001    32        BBFFBX   
         28           3    0  deshd002    12         CFBFH   
         29           3    0  choos001    11           BSX   
         30           3    1  calhk001    32       SBBFBFC   
         31           3    1  troum001    31         BBBCX   
         32           3    1  bourj002    22         BCSBX   
         33           3    1  fletd002    22         CBBCX   
         34           3    1  pujoa001    31         BBFBB   
         35           3    1  lastt001    21          BCBB   
         36           3    1  lastt001    32       BCBB.CX   
         37           4    0  odorr001    32       CSBBBFX   
         38           4    0  andre001    12          SBFX   
         39           4    0  gallj002    10           B1X   
         40           4    1  lucrj001    12          CBFX   
         41           4    1  goodb001    30          BBBB   
         42           4    1  cozaz001    32      BCBC1FBX   
         43           4    1  calhk001    00                 
         44           4    1  calhk001    12         .FFBS   
         45           5    0  cabra002    00             X   
         46           5    0  guzmr001    31         SBBBB   
         47           5    0  forsl001    00                 
         48           5    0  forsl001    21         .BBCX   
         49           5    0  kinei001    12          CBSX   
         50           5    0  deshd002    02           CCX   
         51           5    0  choos001    11          C*BX   
         52           5    0  odorr001    22         BBFSX   
         53           5    1  troum001    22        CCFBBS   
         54           5    1  bourj002    22         BBCCX   
         55           5    1  fletd002    01            CX   
         56           5    1  pujoa001    01            CX   
         57           6    0  andre001    02           CFX   
         58           6    0  gallj002    32       BSBBSFB   
         59           6    0  cabra002    22        CBFBFX   
         60           6    1  lastt001    32       BCCBFBB   
         61           6    1  lucrj001    22      CSBBFFFS   
         62           6    1  goodb001    22     

In [17]:
pd.DataFrame.set_index?

Signature:
pd.DataFrame.set_index(
    self,
    keys,
    drop=True,
    append=False,
    inplace=False,
    verify_integrity=False,
)
Docstring:
Set the DataFrame index using existing columns.

Set the DataFrame index (row labels) using one or more existing
columns or arrays (of the correct length). The index can replace the
existing index or expand on it.

Parameters
----------
keys : label or array-like or list of labels/arrays
    This parameter can be either a single column key, a single array of
    the same length as the calling DataFrame, or a list containing an
    arbitrary combination of column keys and arrays. Here, "array"
    encompasses :class:`Series`, :class:`Index`, ``np.ndarray``, and
    instances of :class:`~collections.abc.Iterator`.
drop : bool, default True
    Delete columns to be used as the new index.
append : bool, default False
    Whether to append columns to existing index.
inplace : bool, default False
    Modify the DataFrame in place (do not create a

In [100]:
re.findall('-H','S8/L.3-H;2-H;1-2'.split("\.")[-1])

['-H', '-H']

In [26]:
re.findall('.-.','5-4')

['5-4']

In [124]:
A = [d['id']]
B = d['plays']['index']
zip_list = zip(cycle([d['id']]), d['plays']['index'])

In [126]:
list(zip_list)

[('BOS201904090', 0),
 ('BOS201904090', 1),
 ('BOS201904090', 2),
 ('BOS201904090', 3),
 ('BOS201904090', 4),
 ('BOS201904090', 5),
 ('BOS201904090', 6),
 ('BOS201904090', 7),
 ('BOS201904090', 8),
 ('BOS201904090', 9),
 ('BOS201904090', 10),
 ('BOS201904090', 11),
 ('BOS201904090', 12),
 ('BOS201904090', 13),
 ('BOS201904090', 14),
 ('BOS201904090', 15),
 ('BOS201904090', 16),
 ('BOS201904090', 17),
 ('BOS201904090', 18),
 ('BOS201904090', 19),
 ('BOS201904090', 20),
 ('BOS201904090', 21),
 ('BOS201904090', 22),
 ('BOS201904090', 23),
 ('BOS201904090', 24),
 ('BOS201904090', 25),
 ('BOS201904090', 26),
 ('BOS201904090', 27),
 ('BOS201904090', 28),
 ('BOS201904090', 29),
 ('BOS201904090', 30),
 ('BOS201904090', 31),
 ('BOS201904090', 32),
 ('BOS201904090', 33),
 ('BOS201904090', 34),
 ('BOS201904090', 35),
 ('BOS201904090', 36),
 ('BOS201904090', 37),
 ('BOS201904090', 38),
 ('BOS201904090', 39),
 ('BOS201904090', 40),
 ('BOS201904090', 41),
 ('BOS201904090', 42),
 ('BOS201904090', 43)